In [12]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm # for displaying progress

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

import gensim.downloader
# embed = gensim.downloader.load("glove-wiki-gigaword-200")

In [11]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [5]:
path = os.getcwd()
base_directory = path + '/Data'

In [6]:
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [2]:
def method1(path, img_name):
    # load the image from file
    image = load_img(path, target_size=(224, 224))
    # convert image pixels to numpy array
    image = img_to_array(image)
    # reshape data for model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # preprocess image for vgg
    image = preprocess_input(image)
    # extract features
    feature = model.predict(image, verbose=0)
    # get image ID
    image_id = img_name.split('.')[0]
    # store feature
    return image_id, image

In [3]:
def method2(path, img_name):
    target_height = 224
    target_width = 224
    # Load and preprocess image
    image = tf.io.read_file(path) 
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # Calculate the aspect ratio of the original image
    original_height, original_width, _ = tf.unstack(tf.shape(image))
    aspect_ratio = tf.cast(original_width, tf.float32) / tf.cast(original_height, tf.float32)

    # Calculate the new dimensions while preserving the aspect ratio
    if aspect_ratio > 1.0:
        new_width = target_width
        new_height = tf.cast(target_width / aspect_ratio, tf.int32)
    else:
        new_height = target_height
        new_width = tf.cast(target_height * aspect_ratio, tf.int32)

    # Resize and pad the image to the target size
    resized_image = tf.image.resize_with_pad(image, target_height, target_width, method='bilinear')

    # Convert the image to grayscale
    resized_image = tf.image.rgb_to_grayscale(resized_image)
    resized_image = tf.image.convert_image_dtype(resized_image, tf.uint8)

    image_id = img_name.split('.')[0]
    return image_id, resized_image

In [26]:
img_features = {}
working_directory = base_directory + '/images'

target_height = 224
target_width = 224

for img_name in os.listdir(working_directory)[:1]:
    if img_name.endswith('.jpg'):
        path = working_directory + '/' + img_name

        imid, fe = method1(path, img_name)
        # imid, fe = method2(path, img_name)
        img_features[imid] = fe

print(list(img_features.values())[0])


[[1.17443975e-07 2.13992166e-06 3.55783953e-07 6.29982679e-08
  1.66249247e-05 3.69458417e-06 4.13247079e-07 9.86036284e-08
  1.10921206e-07 1.02287163e-06 7.99661620e-07 6.54590337e-07
  4.55615634e-07 1.26813973e-06 6.19725597e-06 3.99473862e-07
  9.75871558e-07 1.33811909e-06 7.43665714e-06 2.05460196e-07
  1.27552951e-07 1.37782013e-06 6.27699308e-07 3.98002214e-07
  1.77376052e-07 1.93655751e-06 9.77204854e-06 1.33347758e-05
  2.99444605e-06 5.89237175e-07 2.08197761e-07 6.47326999e-07
  1.03161403e-06 1.11814202e-07 1.39579413e-07 3.12107886e-07
  1.62449183e-07 4.35877539e-08 3.23927054e-07 5.55584094e-08
  4.52517088e-06 1.08116330e-06 6.92668777e-07 4.16327630e-08
  1.01233434e-06 4.20283129e-07 3.48549770e-06 3.30559078e-06
  6.96137192e-08 2.95885826e-07 6.34449719e-08 9.22795437e-07
  6.21178242e-06 2.17436491e-06 1.08204006e-06 2.06616073e-06
  2.61200256e-07 8.31504622e-07 5.39524592e-07 2.17020606e-05
  1.72629257e-06 6.57287913e-08 1.34802704e-07 3.17499712e-07
  8.6898